# doc_to_vector

c.f.) word2vec; sentence -> tokenization -> cleansing, stemming -> encoding -> sorting -> padding/similarity


### 0) 환경 설정
* tokenizer (한글 형태소 분석기) 설치
* 한글 글꼴 설치
* 사전 학습을 위한 한글 말뭉치 패키지 설치
* 설치 후 런타임 재시작 필요

In [ ]:
# tokenizer
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# hangul font
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf


# # Korpora install
# !pip install Korpora


# debug mode set up
# !pip install -Uqq ipdb
## 사용법
# import ipdb  #; ipdb.set_trace()  ipdb > pdb
# %pdb on


# dart.csv 파일 다운로드
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL" -O dart.csv && rm -rf /tmp/cookies.txt


## 한글 doc2vec by gensim
#### reference
* https://colab.research.google.com/github/kiyoungkim1/ReadyToUseNlp/blob/main/notebooks/nlp/doc2vec.ipynb  
* https://wikidocs.net/155356  


In [3]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

'''
file load
'''
df = pd.read_csv('/content/drive/MyDrive/nlp/dart.csv', sep=',')
# df.isnull().sum()
# print('before dropna ', len(df))  # 2589
df = df.dropna()
# print('after dropna ', len(df))  # 2295

'''
exclude special symbols
'''
df['business'] = df['business'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
df['business'] = df['business'].str.replace(" +"," ")

# df = df[:10]
df[:10]

2589
2295


<ipython-input-3-ae040b96a743>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['business'] = df['business'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
<ipython-input-3-ae040b96a743>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df['business'] = df['business'].str.replace(" +"," ")


,code,market,name,business
0,000020,KOSPI,동화약품,사업의 내용 사업의 개요 가 일반적인 사항 기업회계기준서 제 호 연결재무제표 의 ...
1,000040,KOSPI,KR모터스,사업의 내용 사업의 개요 가 업계의 현황 수출주력시장인 유럽 불경기에 따른 대배기...
2,000050,KOSPI,경방,사업의 내용 사업의 개요 산업의 특성 섬유사업부문 면방직산업은 과거 우리나라 경제...
3,000060,KOSPI,메리츠화재,사업의 내용 사업의 개요 가 산업의 특성 성장성 경기변동의 특성 계절성 산업의 특...
4,000070,KOSPI,삼양홀딩스,사업의 내용 사업의 개요 가 업계의 현황 지주회사 란 다른 회사의 주식을 소유함으...
6,000080,KOSPI,하이트진로,사업의 내용 사업의 개요 가 사업부문별 성과 당사 연결실체 의 영업부문은 수익을 ...
8,000100,KOSPI,유한양행,사업의 내용 사업의 개요 가 업계의 현황 산업의 특성 가 국민복지 증진과 직결 국...
10,000120,KOSPI,CJ대한통운,사업의 내용 사업의 개요 가 사업부문별 종속회사 현황 사업부문명 연결회사 및 종속...
11,000140,KOSPI,하이트진로홀딩스,사업의 내용 사업의 개요 당사는 한국채택국제회계기준 적용기업으로 당사의 연결대상 ...
13,000150,KOSPI,두산,사업의 내용 사업의 개요 가 사업부문별 주요 재무현황 사업부문 구성 구분 주요제품...


In [4]:
# business열에 대해서 형태소 분석하고, 학습에 필요한 형식(해당 문서의 '제목' + 단어 토큰화된 상태의 문서의 '본문')으로 변환
# TaggedDocument의 tags에 해당 문서의 '제목'을, words에 해당 문서의 '본문'을 저장하여 리스트를 생성
mecab = Mecab()
tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
    text = row['business']
    tag = row['name']
    # print(index, text, tag)
    tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))

# tagged_corpus_list[0]
print('문서의 수 :', len(tagged_corpus_list))


100%|██████████| 2295/2295 [00:59<00:00, 38.82it/s]

문서의 수 : 2295


In [5]:
'''
doc2vec 학습 및 테스트
'''
from gensim.models import doc2vec
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
# print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')  # for old version gensim==3.8.3
print(f"Tag Size: {len(model.dv.index_to_key)}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
# model.save('/content/drive/MyDrive/nlp/dart.doc2vec')
model.save('dart.doc2vec')


Tag Size: 2295 / 

In [11]:
# 모델 load
model = doc2vec.Doc2Vec.load('/content/drive/MyDrive/nlp/dart.doc2vec')

# similar_doc = model.docvecs.most_similar('동화약품')
# similar_doc = model.docvecs.most_similar('하이트진로')
# similar_doc = model.docvecs.most_similar('LG이노텍')
# similar_doc = model.docvecs.most_similar('메리츠화재')
similar_doc = model.docvecs.most_similar('카카오')

print(similar_doc)

[('NAVER', 0.5165902972221375), ('카카오게임즈', 0.47397923469543457), ('TJ미디어', 0.4524120092391968), ('네오위즈', 0.4516659080982208), ('갤럭시아머니트리', 0.4407119154930115), ('넵튠', 0.43842607736587524), ('베스파', 0.43604350090026855), ('NHN', 0.4335773289203644), ('판도라티비', 0.4200457036495209), ('네오위즈홀딩스', 0.41816937923431396)]


<ipython-input-11-ec61622aef2f>:8: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar('카카오')


In [ ]:
# doc2vec example by https://www.youtube.com/watch?v=5ivVf-Guqk4
# gensim 3.8.3 is not available in colab env.

# !pip install --upgrade -q gensim==3.8.3
# !git clone https://github.com/kiyoungkim1/ReadyToUseAI

# from ReadyToUseAI.src.nlp import make_sample_dataset, doc2vec
# make_sample_dataset.nsmc(mode='test', text_only=True)

# # training
# doc2vec.apply(data_path='dataset.txt', save_name='doc2vec_model', size=150, window=5, min_count=3)

# # # similarity inference
# doc_index = 416
# doc2vec.get_similar_doc(doc_index, save_name='doc2vec_model')

# comment

* word2vec은 자연어 처리의 기본 개념을 파악하는데 도움이 되지만, 단어의 순서를 고려하지 않으므로, 출현하는 단어가 같으면 의미가 반대여도 유사하다고 판단할 수 있음. 즉, 성능면에서 좋은 방법은 아님
* labeled test data가 없어, test data에 대한 평가를 할 수 없음
* 보다비에서 정답 데이터가 [label, text]의 형태로 오면 평가를 할 수 있을 듯
* 학습을 위한 데이터 양이 너무 적음
* 기본적인 개념이 파악되면, doc2vec, kobert 등 다른 방법을 시도해 보는 것이 좋겠음



